# **Second Program**


## Steps:


### video -> HaarCascadeClassifier -> FaceNet (CNN) -> FaceSignature (Matrix 128 X 1)-> compare to database

In [1]:


from PIL import Image

from keras.models import load_model
import numpy as np
from numpy import asarray
from numpy import expand_dims
from keras_facenet import FaceNet

import pickle
import cv2

In [2]:
haar_cascade = cv2.CascadeClassifier("/home/innovalabs/Escritorio/IoT/env/lib/python3.6/site-packages/cv2/data/haarcascade_frontalface_default.xml")
MyFaceNet = FaceNet()

In [3]:
file = open("data.pkl", "rb")
database = pickle.load(file)
file.close()

In [4]:
cap = cv2.VideoCapture(0)
cap.isOpened()


True

In [5]:


while(1):
    _, gbr1 = cap.read()
    face_detection = haar_cascade.detectMultiScale(gbr1,1.1,4)

    if len(face_detection) > 0:
        x1, y1, width, height = face_detection[0]
    else:
        x1, y1, width, height = 1, 1, 10, 10
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height
    
    gbr = cv2.cvtColor(gbr1, cv2.COLOR_BGR2RGB)
    gbr = Image.fromarray(gbr)
    gbr_array = asarray(gbr)
    
    # crop image to only the face
    face = gbr_array[y1:y2, x1:x2]
    
    face = expand_dims(face, axis=0) #this converts from 160 x 160 x 3 to 1 x 160 x 160 x 3, as facenet need 4dimension image
    signature = MyFaceNet.embeddings(face)
    
    min_dist = 100
    identity = ''
    
    for key,value in database.items():
        dist = np.linalg.norm(value-signature)
        if dist < min_dist:
            min_dist = dist
            identity = key
            
    cv2.putText(gbr1,identity, (100,100), cv2.FONT_HERSHEY_SIMPLEX,1, (225,225,0), 2, cv2.LINE_AA)
    cv2.rectangle(gbr1,(x1,y1),(x2,y2),(0,255,0),2)
    
    cv2.imshow('res',gbr1)

    

    k = cv2.waitKey(0)
    if k == 27:         # wait for ESC key to exit
        cv2.destroyAllWindows()

error: OpenCV(4.8.1) /tmp/pip-install-l2pe88r_/opencv-python_5a5bce3f50754bce9f9fea77333e3678/opencv/modules/highgui/src/window.cpp:1272: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'
